In [53]:
import base64
import json

from dotenv import load_dotenv

load_dotenv()

import os

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI

import requests
import pandas as pd


ParserError: Error tokenizing data. C error: Expected 9 fields in line 59, saw 10


In [ ]:
def qa(file_path, prompt):

    loader = TextLoader(file_path)

    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)

    embeddings = OpenAIEmbeddings()
    docsearch = Chroma.from_documents(texts, embeddings)

    qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.as_retriever())

    return qa.run(prompt)

In [ ]:
prompt = """
Fill in X:

The rockies won X games. They lost X games. They played X games.

On average they score X runs per game. The average score of their opponents was X runs per game.

They won X games at home and X games on the road.

The games they won their win probability was X. The games they lost their win probability was X.

Their next game is 7/5/2023 against X. Their win probability is X. They will likely score X runs.

If you don't know, estimate.

"""
rockies_estimate = qa('data/rockies_schedule.csv', prompt)

rockies_estimate

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


'\nThe rockies won 25 games. They lost 24 games. They played 49 games.\n\nOn average they score 6.1 runs per game. The average score of their opponents was 5.6 runs per game.\n\nThey won 17 games at home and 8 games on the road.\n\nThe games they won their win probability was 0.488. The games they lost their win probability was 0.415.\n\nTheir next game is 7/5/2023 against Houston. Their win probability is 0.327. They will likely score 4-5 runs.'

In [ ]:
prompt = """
You are a professional sports gambler. You are trying to predict the outcome of the next game.

Fill in X:

The astros won X games. They lost X games. They played X games.

On average they score X runs per game. The average score of their opponents was X runs per game.

They won X games at home and X games on the road.

The games they won their win probability was X. The games they lost their win probability was X.

Their next game is 7/5/2023 against X. Their win probability is X. They will likely score X runs.

If you don't know, estimate.

"""
astros_estimate = qa('data/astros_schedule.csv', prompt)

astros_estimate

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


' The Astros won 25 games, they lost 19 games, they played 44 games. On average they scored 5.25 runs per game and their opponents scored 3.75 runs per game. They won 16 games at home and 9 games on the road. The games they won their win probability was 0.605 and the games they lost their win probability was 0.441. Their next game is 7/5/2023 against the Colorado Rockies. Their win probability is 0.612 and they will likely score 4 runs.'

In [ ]:
estimate = rockies_estimate + astros_estimate
with open('data/estimate.txt', 'w') as f:
    f.write(estimate)

In [ ]:

prompt = """

Would you take the over or under in this bet: Colorado Rockies Total (ov 9) (-110) vs Astros Total (Un 9) (-110)

Here is a detailed explanation of the bet:

In this bet, you are wagering on the total number of runs scored by both teams combined in the game.

The "ov 9" stands for "over 9" runs. If you bet on this, you're betting that the total combined score of both teams will be more than 9 runs. The "-110" means that you would need to bet $110 to win $100 (plus your original $110 stake back) if the total runs scored is indeed over 9.

On the flip side, "Un 9" stands for "under 9" runs. If you bet on this, you're betting that the total combined score of both teams will be less than 9 runs. Again, the "-110" indicates that you would need to wager $110 to win $100 (plus your original $110 stake back) if the total runs scored is under 9.

If exactly 9 runs are scored, it would be a "push", and your stake would usually be returned to you (it would be like the bet never happened).

"""
winner = qa('data/estimate.txt', prompt)
winner

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


'\nIt is difficult to make a decision without knowing more information, but based on the data provided, I would take the under 9 runs bet. Both teams have an average of 5.45 runs per game combined, which is below 9 runs. Additionally, the Astros win probability of 0.612 indicates that they have a higher chance of winning, which could lead to a lower combined score.'